In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0870, -0.0279,  0.1620,  0.1927, -0.0818, -0.1765,  0.0588,  0.0933,
         -0.2301,  0.1018],
        [-0.0617,  0.0245,  0.2149,  0.1813, -0.1262,  0.0191,  0.0354, -0.0735,
         -0.0946,  0.0705]], grad_fn=<AddmmBackward>)

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [3]:
net = MLP()
net(X)

tensor([[-0.2383, -0.1853, -0.2839,  0.2163, -0.1286, -0.0876, -0.1368, -0.1382,
         -0.1382,  0.1442],
        [-0.2941, -0.1028, -0.1444,  0.1969, -0.0910, -0.0510, -0.1594, -0.0950,
         -0.0451,  0.1174]], grad_fn=<AddmmBackward>)

In [4]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.1024, -0.1844,  0.0680, -0.1850, -0.0534, -0.1400, -0.0697,  0.1163,
         -0.0546,  0.3654],
        [-0.0298, -0.2129, -0.0916, -0.0833, -0.0143, -0.0581, -0.1306,  0.0815,
          0.0153,  0.3838]], grad_fn=<AddmmBackward>)

In [5]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.1238, grad_fn=<SumBackward0>)

In [6]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1308, grad_fn=<SumBackward0>)